In [ ]:
from efaar_benchmarking.data_loading import load_replogle
from efaar_benchmarking.efaar import *
from efaar_benchmarking.constants import *
from efaar_benchmarking.benchmarking import univariate_consistency_benchmark, multivariate_benchmark
from efaar_benchmarking.plotting import plot_recall

recall_threshold_pairs = []
start = 0.01
end = 0.99
step = 0.01

while start <= .105 and end >= .895:
    recall_threshold_pairs.append((round(start,2), round(end,2)))
    start += step
    end -= step

print(recall_threshold_pairs)

In [ ]:
maps2build = ["PCA-CS", "scVI-CS", "scVI-TVN"]
for m in maps2build:
    if m in ["scVI-CS", "scVI-TVN"]:
        adata = load_replogle("genome_wide", "raw") # use for scVI Embeddings
        embeddings = embed_by_scvi_anndata(adata) # scVI Embeddings
        metadata = adata.obs
        fn = tvn_on_controls if "TVN" in m else centerscale_on_controls
        embeddings = fn(embeddings, metadata, pert_col=REPLOGLE_PERT_LABEL_COL, control_key=REPLOGLE_CONTROL_PERT_LABEL)
    else:
        adata = load_replogle("genome_wide", "normalized") # use for PCA Embeddings
        embeddings = embed_by_pca_anndata(adata) # PCA Embeddings
        metadata = adata.obs
        embeddings = centerscale_on_controls(embeddings, metadata, pert_col=REPLOGLE_PERT_LABEL_COL, control_key=REPLOGLE_CONTROL_PERT_LABEL)
    # consistency_pvals = univariate_consistency_benchmark(embeddings, metadata, pert_col=REPLOGLE_PERT_LABEL_COL, keys_to_drop=[REPLOGLE_CONTROL_PERT_LABEL])
    map_data = aggregate(embeddings, metadata, pert_col=REPLOGLE_PERT_LABEL_COL, control_key=REPLOGLE_CONTROL_PERT_LABEL)
    for right_sided in [False]:
        metrics = multivariate_benchmark(map_data, recall_thr_pairs=recall_threshold_pairs, pert_col=REPLOGLE_PERT_LABEL_COL, n_null_samples = 10000, n_iterations = 1, right_sided=right_sided)
        plot_recall({f'GWPS {m}': metrics}, right_sided=right_sided, title="Right tail only" if right_sided else "Both tails")
        print(metrics.groupby('source')['recall_0.05_0.95'].mean())